# How to use a AutoML Video  object tracking model for batch prediction

##### Copyright
**GUI Author:** [ Google Cloud](https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/automl/sdk_automl_video_object_tracking_batch.ipynb)<br>
**Notebook Adoption Author:** [Paul Kamau](https://paulkamau.com)<br>
**Project Type:** AutoML Object Detection Video<br>
**Date created:** 2022/12/10<br>
**Dataset** The dataset used for this tutorial is the [Traffic](https://storage.googleapis.com/automl-video-demo-data/traffic_videos/traffic_videos_labels.csv) dataset. The version of the dataset you will use in this tutorial is stored in a public Cloud Storage bucket.

This project demonstrates how to use the Vertex AI SDK for Python to create video object tracking models and do batch prediction using a Google Cloud [AutoML](https://cloud.google.com/vertex-ai/docs/start/automl-users) model.

### Objective

- Create an AutoML video object tracking model from a Python script
- Create a Vertex `Dataset` resource.
- Train the model.
- View the model evaluation.
- Make a batch prediction.

### Dataset

The dataset used for this tutorial is the [Traffic](https://storage.googleapis.com/automl-video-demo-data/traffic_videos/traffic_videos_labels.csv) dataset. The version of the dataset you will use in this tutorial is stored in a public Cloud Storage bucket.

[![Colab](https://cloud.google.com/ml-engine/images/colab-logo-32px.png) Run in colab](https://colab.research.google.com/github/)

# Envrionment prep

Install the latest version of Vertex SDK for Python.

In [1]:
import os

# Google Cloud Notebook
if os.path.exists("/opt/deeplearning/metadata/env_version"):
    USER_FLAG = "--user"
else:
    USER_FLAG = ""

#### Import the Google Cloud SDK packages

In [8]:
! pip3 install --upgrade google-cloud-aiplatform $USER_FLAG "shapely<2"
! pip3 install -U google-cloud-storage $USER_FLAG

import json
import pandas as pd
import google.cloud.aiplatform as aiplatform
from google.cloud import storage

### Restart the kernel

Once you've installed the additional packages, you need to restart the notebook kernel so it can find the packages.

In [2]:

if not os.getenv("IS_TESTING"):
    # Automatically restart kernel after installs
    import IPython

    app = IPython.Application.instance()
    app.kernel.do_shutdown(True)

## Before you begin

### GPU runtime

This tutorial does not require a GPU runtime.

### Set up your Google Cloud project

**The following steps are required, regardless of your notebook environment.**

1. [Select or create a Google Cloud project](https://console.cloud.google.com/cloud-resource-manager). When you first create an account, you get a $300 free credit towards your compute/storage costs.

2. [Make sure that billing is enabled for your project.](https://cloud.google.com/billing/docs/how-to/modify-project)

3. [Enable the following APIs: Vertex AI APIs, Compute Engine APIs, and Cloud Storage.](https://console.cloud.google.com/flows/enableapi?apiid=ml.googleapis.com,compute_component,storage-component.googleapis.com)

4. If you are running this notebook locally, you will need to install the [Cloud SDK]((https://cloud.google.com/sdk)).

5. Enter your project ID in the cell below. Then run the  cell to make sure the
Cloud SDK uses the right project for all the commands in this notebook.

**Note**: Jupyter runs lines prefixed with `!` as shell commands, and it interpolates Python variables prefixed with `$`.

## Project variables
These are the project variable used in this ML Model:

In [9]:
# Project variables
#
# These are the project variable used in this ML Model:
#
PROJECT_ID = "" # @param {type:"string"}
REGION = "us-central1" #@param ["us-central1", "europe-west4", "asia-east1", "video"]
automl_type = "video" #@param ["image", "text", "tabular", "video"]
model_display_name = "automl_video_traffic_detection_model"
job_display_name = model_display_name + "_job"
endpoint_display_name = model_display_name + "_endpoint"

## datasets
dataset_display_name = model_display_name + "_dataset"
dataset_source_uri = "gs://automl-video-demo-data/traffic_videos/traffic_videos_labels.csv"
dataset_source_public = "https://storage.googleapis.com/automl-video-demo-data/traffic_videos/traffic_videos_labels.csv"
batch_display_name = model_display_name + "_batch_prediction"

# bucket details
BUCKET_NAME = "auto-ml-tutorials" #auto-ml bucket
BUCKET_URI = f"gs://{BUCKET_NAME}/{automl_type}/" # automl bucket uri
BUCKET_PREDICTION_OUTPUT = f"gs://auto_ml_datasets_predictions/{automl_type}/"
BUCKET_INPUT_BATCHPREDICT = f"gs://{BUCKET_NAME}/{automl_type}/{model_display_name}" # contains the files to be used for batch prediction

print("All project variables set. Lets go")

All project variables set. Lets go


In [4]:
# If you are running this notebook in Colab, run this cell and follow the
# instructions to authenticate your GCP account. This provides access to your
# Cloud Storage bucket and lets you submit training jobs and prediction
# requests.

import os
import sys

# If on Google Cloud Notebook, then don't execute this code
if not os.path.exists("/opt/deeplearning/metadata/env_version"):
    if "google.colab" in sys.modules:
        from google.colab import auth as google_auth

        google_auth.authenticate_user()

    # If you are running this notebook locally, replace the string below with the
    # path to your service account key and run this cell to authenticate your GCP
    # account.
    elif not os.getenv("IS_TESTING"):
        %env GOOGLE_APPLICATION_CREDENTIALS ''

### Create a Cloud Storage bucket

**The following steps are required, regardless of your notebook environment.**

When you initialize the Vertex SDK for Python, you specify a Cloud Storage staging bucket. The staging bucket is where all the data associated with your dataset and model resources are retained across sessions.

Set the name of your Cloud Storage bucket below. Bucket names must be globally unique across all Google Cloud projects, including those outside of your organization.

**Only if your bucket doesn't already exist**: Run the following cell to create your Cloud Storage bucket.

In [ ]:
!gsutil mb -l $REGION -p $PROJECT_ID $BUCKET_URI

Finally, validate access to your Cloud Storage bucket by examining its contents:

In [5]:
!gsutil ls -al $BUCKET_URI

         0  2022-07-23T20:52:09Z  gs://auto-ml-tutorials/video/#1658609529122789  metageneration=1
     54106  2022-07-23T20:52:58Z  gs://auto-ml-tutorials/video/hmdb_split1_5classes_all.csv#1658609578821711  metageneration=1
TOTAL: 2 objects, 54106 bytes (52.84 KiB)


## Initialize Vertex SDK for Python

Initialize the Vertex SDK for Python for your project and corresponding bucket.

In [5]:
aiplatform.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

# Build the Machine Learning Model

Now you are ready to start creating your own AutoML video object tracking model.

#### Location of Cloud Storage training data.

Now set the variable `dataset_source_uri` to the location of the CSV index file in Cloud Storage.

In [10]:
dataset_source_uri = dataset_source_uri

#### Quick peek at your data

This tutorial uses a version of the Traffic dataset that is stored in a public Cloud Storage bucket, using a CSV index file.

Start by doing a quick peek at the data. You count the number of examples by counting the number of rows in the CSV index file  (`wc -l`) and then peek at the first few rows.

In [69]:
data_rows_count = ! gsutil cat $dataset_source_uri | wc -l
print(f"total number of rows = {data_rows_count}")

# Read the data
dataset_sample = pd.read_csv(dataset_source_public,nrows=5)

# Use the next code cell to print the first five rows of the data.
dataset_sample.head()

print("\n5 rows sample")
print(dataset_sample.to_string())
#print(dataset_sample.head())

total number of rows = ['685']

5 rows sample
   gs://automl-video-demo-data/traffic_videos/highway_005.mp4           sedan  1565750291672021  11.933333  0.509205  0.594283  Unnamed: 6  Unnamed: 7  0.728737  0.760959  Unnamed: 10  Unnamed: 11
0  gs://automl-video-demo-data/traffic_videos/highway_005.mp4  pickup_suv_van  1565750291672171  17.566666  0.761241  0.498466         NaN         NaN  0.948839  0.668524          NaN          NaN
1  gs://automl-video-demo-data/traffic_videos/highway_005.mp4  pickup_suv_van  1565750291672223  20.433333  0.000000  0.465235         NaN         NaN  0.142638  0.665644          NaN          NaN
2  gs://automl-video-demo-data/traffic_videos/highway_005.mp4  pickup_suv_van  1565750291672347  25.766666  0.486523  0.592331         NaN         NaN  0.720611  0.776687          NaN          NaN
3  gs://automl-video-demo-data/traffic_videos/highway_005.mp4  pickup_suv_van  1565750291672575  28.966666  0.578534  0.652778         NaN         NaN  0.828647  0.86

In [67]:
## list columns
print("\nThese are the columns")
cols = list(pd.read_csv(dataset_source_public, nrows =1))
for i in cols:
       print(i)

# Shape of training data (num_rows, num_columns)
#print(dataset_sample.head())


These are the columns
gs://automl-video-demo-data/traffic_videos/highway_005.mp4
sedan
1565750291672021
11.933333
0.509205
0.594283
Unnamed: 6
Unnamed: 7
0.728737
0.760959
Unnamed: 10
Unnamed: 11


#### Here's a sample of the video

Let's view a sample of the video using the function below.

In [24]:
from IPython.display import HTML

# enter the video url in the variable below after '''https://storage.googleapis.com/'''
video_url = "https://storage.googleapis.com/automl-video-demo-data/traffic_videos/highway_005.mp4"

HTML(f"""
<video width="320" height="240" controls>
  <source src="{video_url}" "video/mp4">
</video>
""")

### Create the Dataset

Next, create the `Dataset` resource using the `create` method for the `VideoDataset` class, which takes the following parameters:

- `display_name`: The human readable name for the `Dataset` resource.
- `gcs_source`: A list of one or more dataset index files to import the data items into the `Dataset` resource.

This operation may take several minutes.

In [25]:
## Create dataset if it doesn't exist
datasets = aiplatform.VideoDataset.list(filter = f"display_name={dataset_display_name}")

if datasets:
    dataset = datasets[0]
    print(f"Dataset Exists: {datasets[0].display_name}")
else:
    dataset = aiplatform.ImageDataset.create (
    display_name=dataset_display_name,
    gcs_source=[dataset_source_uri],
    import_schema_uri= aiplatform.schema.dataset.ioformat.video.object_tracking,
    )
    print(f"Dataset Created: {dataset.display_name}")

print(f'Review the Dataset in the Console:\nhttps://console.cloud.google.com/vertex-ai/locations/{REGION}/datasets/{datasets[0].resource_name}?project={PROJECT_ID}')


Dataset Exists: automl_video_traffic_detection_model_dataset
Review the Dataset in the Console:
https://console.cloud.google.com/vertex-ai/locations/us-central1/datasets/projects/993987777814/locations/us-central1/datasets/928500186261487616?project=paulkamau


### Create and run training pipeline

To train an AutoML model, you perform two steps: 1) create a training pipeline, and 2) run the pipeline.

#### Create training pipeline

An AutoML training pipeline is created with the `AutoMLVideoTrainingJob` class, with the following parameters:

- `display_name`: The human readable name for the `TrainingJob` resource.
- `prediction_type`: The type task to train the model for.
  - `classification`: A video classification model.
  - `object_tracking`: A video object tracking model.
  - `action_recognition`: A video action recognition model.


In [26]:
# Define the training job and create one if it doesn't exit
jobs = aiplatform.AutoMLVideoTrainingJob.list(filter = f"display_name={job_display_name}")

if jobs:
    job = jobs[0]
    print(f"Jobs Exists: {jobs[0].display_name}")
else:
    job = aiplatform.AutoMLImageTrainingJob(
    display_name=job_display_name,
    prediction_type="object_tracking",
    )
    print(f"job Created: {job.display_name}")

print(f'Review the job in the Console:\nhttps://console.cloud.google.com/vertex-ai/locations/{REGION}/jobs/{jobs[0].resource_name}?project={PROJECT_ID}')


Jobs Exists: automl_video_traffic_detection_model_job
Review the job in the Console:
https://console.cloud.google.com/vertex-ai/locations/us-central1/jobs/projects/993987777814/locations/us-central1/trainingPipelines/6035150880263634944?project=paulkamau


#### Run the training pipeline

Next, you start the training job by invoking the method `run`, with the following parameters:

- `dataset`: The `Dataset` resource to train the model.
- `model_display_name`: The human readable name for the trained model.
- `training_fraction_split`: The percentage of the dataset to use for training.
- `test_fraction_split`: The percentage of the dataset to use for test (holdout data).

The `run` method when completed returns the `Model` resource.

The execution of the training pipeline will take upto 5 hours.

In [43]:
# Create model job if it doesn't exist
models = aiplatform.Model.list(filter = f"display_name={model_display_name}")

if models:
    model = models[0]
    print(f"Model Exists: {models[0].display_name}")
else:
    model = job.run(
    dataset=dataset,
    model_display_name=model_display_name,
    training_fraction_split=0.8,
    test_fraction_split=0.2,
    model_registry = "vertex_ai",
    )
    print(f"Model created: {models[0].display_name}")

print(f'Review the Model in the Console:\nhttps://console.cloud.google.com/vertex-ai/locations/{REGION}/models/{model.name}?project={PROJECT_ID}')


Model Exists: automl_video_traffic_detection_model
Review the Model in the Console:
https://console.cloud.google.com/vertex-ai/locations/us-central1/models/7844632834135293952?project=paulkamau


## Review model evaluation scores
After your model has finished training, you can review the evaluation scores for it.

First, you need to get a reference to the new model. As with datasets, you can either use the reference to the model variable you created when you deployed the model or you can list all of the models in your project.

In [56]:
# Model Evaluations
model_evaluations = model.list_model_evaluations()
model_evaluation = list(model_evaluations)[0]
print(model_evaluation)


# Print the evaluation metrics
for evaluation in model_evaluations:
    evaluation = evaluation.to_dict()
    print("Model's evaluation metrics from Training:\n")
    metrics = evaluation["metrics"]
    for metric in metrics.keys():
        print(f"metric: {metric}, value: {metrics[metric]}\n")

resource name: projects/993987777814/locations/us-central1/models/7844632834135293952@1/evaluations/5156113667686662144
Model's evaluation metrics from Training:

metric: boundingBoxMetrics, value: [{'meanAveragePrecision': 0.29013765, 'confidenceMetrics': [{'f1Score': 0.4169279, 'precision': 0.26336634, 'recall': 1.0}, {'confidenceThreshold': 0.024151346, 'recall': 0.18045112, 'f1Score': 0.09073724, 'precision': 0.060606062}, {'recall': 0.18045112, 'precision': 0.060759492, 'confidenceThreshold': 0.02449486, 'f1Score': 0.09090909}, {'recall': 0.18045112, 'precision': 0.060913704, 'f1Score': 0.0910816, 'confidenceThreshold': 0.02516149}, {'f1Score': 0.091254756, 'confidenceThreshold': 0.026261719, 'recall': 0.18045112, 'precision': 0.061068702}, {'confidenceThreshold': 0.02633714, 'precision': 0.06122449, 'recall': 0.18045112, 'f1Score': 0.09142857}, {'confidenceThreshold': 0.02729636, 'recall': 0.18045112, 'precision': 0.061381076, 'f1Score': 0.091603056}, {'precision': 0.06153846, 'c

## Send a batch prediction request

Send a batch prediction to your deployed model.

### Get test item(s)

Now do a batch prediction to your Vertex model. You will use arbitrary examples out of the dataset as a test items. Don't be concerned that the examples were likely used in training the model -- we just want to demonstrate how to make a prediction.

In [57]:
test_items = ! gsutil cat $dataset_source_uri | head -n2
cols_1 = test_items[0].split(",")
cols_2 = test_items[1].split(",")
if len(cols_1) > 12:
    test_item_1 = str(cols_1[1])
    test_item_2 = str(cols_2[1])
    test_label_1 = str(cols_1[2])
    test_label_2 = str(cols_2[2])
else:
    test_item_1 = str(cols_1[0])
    test_item_2 = str(cols_2[0])
    test_label_1 = str(cols_1[1])
    test_label_2 = str(cols_2[1])

In [70]:
# Read the data
dataset_sample = pd.read_csv(dataset_source_public)

## read the last 2 rows
dataset_sample = dataset_sample.iloc[-2:]

# Use the next code cell to print the first five rows of the data.
dataset_sample.head()

,gs://automl-video-demo-data/traffic_videos/highway_005.mp4,sedan,1565750291672021,11.933333,0.509205,0.594283,Unnamed: 6,Unnamed: 7,0.728737,0.760959,Unnamed: 10,Unnamed: 11
682,gs://automl-video-demo-data/traffic_videos/hig...,sedan,1565718790735127,6.033333,0.0,0.671533,NaN,NaN,0.058931,0.803206,NaN,NaN
683,gs://automl-video-demo-data/traffic_videos/hig...,pickup_suv_van,1565718790735325,9.200000,0.0,0.622585,NaN,NaN,0.099136,0.816469,NaN,NaN


### Make a batch input file

Now make a batch input file, which you store in your local Cloud Storage bucket. The batch input file can be either CSV or JSONL. You will use JSONL in this tutorial. For JSONL file, you make one dictionary entry per line for each video. The dictionary contains the key/value pairs:

- `content`: The Cloud Storage path to the video.
- `mimeType`: The content type. In our example, it is a `avi` file.
- `timeSegmentStart`: The start timestamp in the video to do prediction on. *Note*, the timestamp must be specified as a string and followed by s (second), m (minute) or h (hour).
- `timeSegmentEnd`: The end timestamp in the video to do prediction on.

In [75]:
test_filename = f"{model_display_name}_test_file.jsonl"
gcs_input_uri = BUCKET_INPUT_BATCHPREDICT + f"/{test_filename}"

# making data_1 and data_2 variables using the structure mentioned above
data_1 = {
    "content": test_item_1,
    "mimeType": "video/avi",
    "timeSegmentStart": "0.0s",
    "timeSegmentEnd": "5.0s",
}

data_2 = {
    "content": test_item_2,
    "mimeType": "video/avi",
    "timeSegmentStart": "0.0s",
    "timeSegmentEnd": "5.0s",
}

# getting reference to bucket
bucket = storage.Client(project=PROJECT_ID).bucket(BUCKET_INPUT_BATCHPREDICT)

# creating a blob
blob = bucket.blob(blob_name=test_filename)

# creating data variable
data = json.dumps(data_1) + "\n" + json.dumps(data_2) + "\n"

# uploading data variable content to bucket
blob.upload_from_string(data)

# printing path of uploaded file
print(gcs_input_uri)

# printing content of uploaded file
! gsutil cat $gcs_input_uri

NotFound: ignored

### Make the batch prediction request

Now that your Model resource is trained, you can make a batch prediction by invoking the batch_predict() method, with the following parameters:

- `job_display_name`: The human readable name for the batch prediction job.
- `gcs_source`: A list of one or more batch request input files.
- `gcs_destination_prefix`: The Cloud Storage location for storing the batch prediction resuls.
- `sync`: If set to True, the call will block while waiting for the asynchronous batch job to complete.

In [78]:
# confirm if job exists before running it
jobs = aiplatform.BatchPredictionJob.list(filter = f"display_name={batch_display_name}")

if jobs:
    batch_predict_job = jobs[0]
    print(f"Jobs Exists: {jobs[0].display_name}")
else:
    batch_predict_job = model.batch_predict(
    job_display_name=batch_display_name,
    gcs_source=dataset_source_uri,
    gcs_destination_prefix=BUCKET_PREDICTION_OUTPUT,
    sync=False,
)


Creating BatchPredictionJob


INFO:google.cloud.aiplatform.jobs:Creating BatchPredictionJob


### Wait for completion of batch prediction job

Next, wait for the batch job to complete. Alternatively, one can set the parameter `sync` to `True` in the `batch_predict()` method to block until the batch prediction job is completed.

In [79]:
batch_predict_job.wait()

BatchPredictionJob projects/993987777814/locations/us-central1/batchPredictionJobs/2111900290629238784 current state:
JobState.JOB_STATE_FAILED


INFO:google.cloud.aiplatform.jobs:BatchPredictionJob projects/993987777814/locations/us-central1/batchPredictionJobs/2111900290629238784 current state:
JobState.JOB_STATE_FAILED


RuntimeError: ignored

### Get the predictions

Next, get the results from the completed batch prediction job.

The results are written to the Cloud Storage output bucket you specified in the batch prediction request. You call the method iter_outputs() to get a list of each Cloud Storage file generated with the results. Each file contains one or more prediction requests in a JSON format:

- `content`: The prediction request.
- `prediction`: The prediction response.
 - `id`: The internal assigned unique identifiers for each prediction request.
 - `displayName`: The class names for the predicted label.
 - `confidences`: The predicted confidence, between 0 and 1, per class label.
 - `timeSegmentStart`: The time offset in the video to the start of the video sequence.
 - `timeSegmentEnd`: The time offset in the video to the end of the video sequence.
 - `frames`: Location with frames of the tracked object.

In [ ]:
bp_iter_outputs = batch_predict_job.iter_outputs()

prediction_results = list()
for blob in bp_iter_outputs:
    if blob.name.split("/")[-1].startswith("prediction"):
        prediction_results.append(blob.name)

tags = list()
for prediction_result in prediction_results:
    gfile_name = f"gs://{bp_iter_outputs.bucket.name}/{prediction_result}".replace(
        BUCKET_URI + "/", ""
    )
    data = bucket.get_blob(gfile_name).download_as_string()
    data = json.loads(data)
    print(data)

# Cleaning up

To clean up all Google Cloud resources used in this project, you can [delete the Google Cloud
project](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects) you used for the tutorial.

Otherwise, you can delete the individual resources you created in this tutorial:

- Dataset
- Pipeline
- Model
- Endpoint
- AutoML Training Job
- Batch Job
- Custom Job
- Hyperparameter Tuning Job
- Cloud Storage Bucket

In [ ]:
# Delete the dataset using the Vertex dataset object
dataset.delete()

# Delete the model using the Vertex model object
model.delete()

# Delete the AutoML or Pipeline training job
job.delete()

# Delete the batch prediction job using the Vertex batch prediction object
batch_predict_job.delete()

if os.getenv("IS_TESTING"):
    ! gsutil -m rm -r $BUCKET_URI